#  Wildfire Fire Detection from Satellite Imagery  (UNET)

## Import libraries

In [2]:
#Import Libraries
import os
import sys
import time
import pandas as pd
import numpy as np
import cv2 
import matplotlib.pyplot as plt

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
# Add the path to the functions directory
sys.path.append('../functions')  # Add the path to the functions directory

## Load Dataset

### Load tiles from ../dataset/tiles

### Load tile masks ../dataset/masks

## Split into Training and Validation sets

In [ ]:
# Import library
from sklearn.model_selection import train_test_split

In [ ]:
# Convert tiles to np array
tiles_array = np.stack([np.array(tiles) for tile in tiles]

In [ ]:
# Convert tiles to np array
masks_array = np.stack([np.array(masks) for mask in masks]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(tiles_array, masks_array, test_size=0.2, random_state=42)

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

In [ ]:
# check data types
print(f'Type of X_train {type(X_train)}')
print(f'Type of X_val {type(X_val)}')
print(f'Type of y_train {type(y_train)}')
print(f'Type of y_val {type(y_val)}')

#### Note:
<p> 
    - X_train and y_train contains the training data <br> 
    - X_val and y_val contains the validation data
</p>

In [ ]:
### Plot a random tile

plt.imshow(tiles_array[7])
plt.title('Tile')
plt.axis('off')
plt.show()

## UNET Model design

### Setup Early  Earlystopping and Checkpoints

In [ ]:
# Define directory to save the checkpoints
checkpoint_dir = '../checkpoints'

# directory check
os.makedirs(checkpoint_dir, exist_ok=True)

# checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'unet_checkpoint.keras'),
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Define the early stopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [4]:
from unet import unet

### Build Model

In [6]:
unet_model = unet()

### Compile Model

In [7]:
# Compile the model with an appropriate loss function for multiclass segmentation
unet_model = unet()
unet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Summary

In [8]:
unet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_15 (Conv2D)          (None, 512, 512, 32)         896       ['input_2[0][0]']             
                                                                                                  
 conv2d_16 (Conv2D)          (None, 512, 512, 32)         9248      ['conv2d_15[0][0]']           
                                                                                                  
 max_pooling2d_2 (MaxPoolin  (None, 256, 256, 32)         0         ['conv2d_16[0][0]']           
 g2D)                                                                                       

### Fit Model

In [ ]:
unet_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint_callback, early_stopping_callback],
    verbose=1
)

### Training split 

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(tiles_array, masks_array, test_size=0.3)

### Run Model Evaluation

In [ ]:
score = unet_model.evaluate(test_data, test_labels, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Run  Model predictions

In [ ]:
predictions = unet_model.predict(test_data)

In [ ]:
'''
    View datatypes and information of Predictions  
'''
#test
print(test_preds.shape)  # Check the shape
print(test_preds.dtype)  # Check the data type

In [ ]:
#get first 10 images, view input and ouput images on every line
test_preds = unet_model.predict(test_data[0:8])